In [22]:
import json
import requests
import modland
from VNP21A1D_002 import *

In [15]:
CMR_search_URL = CMR_SEARCH_URL
CMR_search_URL

'https://cmr.earthdata.nasa.gov/search/'

In [3]:
concept_ID = VNP21A1D_002_CONCEPT_ID
concept_ID

'C2545314555-LPCLOUD'

In [8]:
h = 8
v = 5
tile_size = 1200

In [18]:
start_date = "2024-11-11"
end_date = "2024-11-11"

In [10]:
geometry = modland.generate_modland_grid(h=h, v=v, tile_size=tile_size)
geometry

{
  "dimensions": {
    "rows": 1200,
    "cols": 1200
  },
  "bbox": {
    "xmin": -11119505.197665555,
    "ymin": 3335851.5592996655,
    "xmax": -10007554.677899,
    "ymax": 4447802.079066221
  },
  "crs": "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +R=6371007.181 +units=m +no_defs +type=crs",
  "resolution": {
    "cell_width": 926.6254331387962,
    "cell_height": -926.6254331387962
  }
}

In [13]:
centroid = geometry.centroid_latlon
lat = centroid.y
lon = centroid.x
lat, lon

(35.00000000000101, -115.97358593234311)

In [16]:
granule_search_URL = f"{CMR_search_URL}granules.json"
granule_search_URL

'https://cmr.earthdata.nasa.gov/search/granules.json'

In [19]:
params = {
    "concept_id": concept_ID,
    "bounding_box": f"{lon},{lat},{lon},{lat}",  # Point search using the tile's centroid
    "temporal": f"{start_date}T00:00:00Z,{end_date}T23:59:59Z",  # ISO 8601 format for date/time
    "page_size": 2000  # Retrieve up to 2000 granules per page
}

params

{'concept_id': 'C2545314555-LPCLOUD',
 'bounding_box': '-115.97358593234311,35.00000000000101,-115.97358593234311,35.00000000000101',
 'temporal': '2024-11-11T00:00:00Z,2024-11-11T23:59:59Z',
 'page_size': 2000}

In [20]:
response = requests.get(granule_search_URL, params=params)
response

<Response [200]>

In [23]:
data = response.json()

with open("example_CMR_result.json", "w") as f:
    f.write(json.dumps(data, indent=2))

data

{'feed': {'updated': '2024-11-25T18:45:30.616Z',
  'id': 'https://cmr.earthdata.nasa.gov:443/search/granules.json?concept_id=C2545314555-LPCLOUD&bounding_box=-115.97358593234311%2C35.00000000000101%2C-115.97358593234311%2C35.00000000000101&temporal=2024-11-11T00%3A00%3A00Z%2C2024-11-11T23%3A59%3A59Z&page_size=2000',
  'title': 'ECHO granule metadata',
  'entry': [{'producer_granule_id': 'VNP21A1D.A2024316.h08v05.002.2024320160736',
    'time_start': '2024-11-11T00:00:00.000Z',
    'updated': '2024-11-15T10:17:20.614Z',
    'dataset_id': 'VIIRS/NPP Land Surface Temperature/Emissivity Daily L3 Global 1km SIN Grid Day V002',
    'data_center': 'LPCLOUD',
    'title': 'VNP21A1D.A2024316.h08v05.002.2024320160736',
    'coordinate_system': 'GEODETIC',
    'day_night_flag': 'BOTH',
    'time_end': '2024-11-11T23:59:59.000Z',
    'id': 'G3296774315-LPCLOUD',
    'original_format': 'ECHO10',
    'granule_size': '2.3955039978027344',
    'browse_flag': True,
    'polygons': [['29.8308 -115.3696 

In [24]:
URLs = []

for entry in data.get('feed', {}).get('entry', []):  # Safely navigate the JSON structure
    for link in entry.get('links', []):
        URL = link.get('href')
        # Filter for URLs that start with "https" and end with specific file extensions
        if URL and URL.startswith("https") and URL.endswith((".h5")):  
            URLs.append(URL)

URLs

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP21A1D.002/VNP21A1D.A2024316.h08v05.002.2024320160736/VNP21A1D.A2024316.h08v05.002.2024320160736.h5']